In [12]:
#Shortest Remaining Time First
def srtf(processes):
    n = len(processes)
    burst_remaining = [processes[i][1] for i in range(n)]
    arrival_time = [processes[i][0] for i in range(n)]
    completion_time = [0] * n
    turnaround_time = [0] * n
    waiting_time = [0] * n
    total_waiting_time = 0
    total_turnaround_time = 0
    t = 0
    completed = 0
    prev = -1
    gantt_chart = []
    while (completed != n):
        smallest = -1
        for i in range(n):
            if arrival_time[i] <= t and burst_remaining[i] > 0:
                if smallest == -1:
                    smallest = i
                elif burst_remaining[i] < burst_remaining[smallest]:
                    smallest = i
        if smallest == -1:
            t += 1
            continue
        if smallest != prev:
            gantt_chart.append((t, smallest))
        burst_remaining[smallest] -= 1
        t += 1
        if burst_remaining[smallest] == 0:
            completion_time[smallest] = t
            turnaround_time[smallest] = completion_time[smallest] - arrival_time[smallest]
            waiting_time[smallest] = turnaround_time[smallest] - processes[smallest][1]
            total_waiting_time += waiting_time[smallest]
            total_turnaround_time += turnaround_time[smallest]
            completed += 1
            prev = -1
            if gantt_chart[-1][1] != smallest:
                gantt_chart.append((t-1, gantt_chart[-1][1]))
        else:
            prev = smallest
    gantt_chart.append((t, -1))
    print("\n")
    print("Process ID\tAT\tBT\tCT\tTAT\tWT")
    for i in range(n):
        print(f"{i+1}\t\t{arrival_time[i]}\t{processes[i][1]}\t{completion_time[i]}\t{turnaround_time[i]}\t{waiting_time[i]}")
    print("\n")
    print("Gantt Chart:")
    for i in range(len(gantt_chart)-1):
        print(f"{gantt_chart[i][0]}->P{gantt_chart[i][1]+1}", end="")
        if gantt_chart[i][1] != gantt_chart[i+1][1]:
            print(f"->", end="")
    print(f"{gantt_chart[-1][0]}")
    print("\n")
    avg_turnaround_time = total_turnaround_time / n
    avg_waiting_time = total_waiting_time / n
    print(f"Average Turnaround Time: {avg_turnaround_time}")
    print(f"Average Waiting Time: {avg_waiting_time}")



n = int(input("Enter the no. of process: "))
processes = []
for i in range(n):
    at, bt = map(int, input(f"Enter the arrival time & burst time for process P{i+1}: ").split())
    processes.append((at, bt))
srtf(processes)
#{gantt_chart[i+1][0]}



Process ID	AT	BT	CT	TAT	WT
1		0	7	19	19	12
2		1	5	13	12	7
3		2	3	6	4	1
4		3	1	4	1	0
5		4	2	9	5	3
6		5	1	7	2	1


Gantt Chart:
0->P1->1->P2->2->P3->3->P4->4->P3->6->P6->7->P5->9->P2->13->P1->19


Average Turnaround Time: 7.166666666666667
Average Waiting Time: 4.0


In [11]:
#Round Robin
class Process:
    def __init__(self, name, arrival_time, burst_time):
        self.name = name
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.completion_time = 0
        self.turnaround_time = 0
        self.waiting_time = 0
        self.response_time = -1  # Initialize response time to -1
    
    def __repr__(self):
        return self.name


def round_robin(processes, time_quantum):
    n = len(processes)
    current_time = 0
    remaining_processes = processes.copy()
    completed_processes = []
    time_chart = []

    while remaining_processes:
        current_process = remaining_processes.pop(0)

        # Record response time when the process is first executed
        if current_process.response_time == -1:
            current_process.response_time = current_time - current_process.arrival_time

        # Execute the process for the time quantum or until completion
        if current_process.remaining_time > time_quantum:
            time_chart.append((current_process, current_time))
            current_time += time_quantum
            current_process.remaining_time -= time_quantum
            remaining_processes.append(current_process)
        else:
            time_chart.append((current_process, current_time))
            current_time += current_process.remaining_time
            current_process.remaining_time = 0
            current_process.completion_time = current_time
            current_process.turnaround_time = current_process.completion_time - current_process.arrival_time
            current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
            completed_processes.append(current_process)

    avg_turnaround_time = sum(p.turnaround_time for p in completed_processes) / n
    avg_waiting_time = sum(p.waiting_time for p in completed_processes) / n

    # Print process table
    print("{:<8} {:<8} {:<8} {:<12} {:<12} {:<10} {:<8}".format(
        'Process', 'Arrival', 'Burst', 'Completion', 'Turnaround', 'Waiting', 'Response'))
    for p in processes:
        print("{:<8} {:<8} {:<8} {:<12} {:<12} {:<10} {:<10}".format(
            p.name, p.arrival_time, p.burst_time, p.completion_time,
            p.turnaround_time, p.waiting_time, p.response_time))

    # Print average turnaround time and average waiting time
    print("\nAverage Turnaround Time:", avg_turnaround_time, "ms")
    print("Average Waiting Time:", avg_waiting_time, "ms")

    # Print Gantt chart
    print("\nGantt Chart:")
    for i in range(len(time_chart) - 1):
        print(str(time_chart[i][1]) + "->" + str(time_chart[i][0]) + "->", end="")
    print(str(time_chart[-1][1]) + "->" + str(time_chart[-1][0]) + "->" + str(time_chart[-1][1] + time_quantum))


# Example usage
n = int(input("Enter the number of processes: "))
processes = []
for i in range(n):
    at, bt = map(int, input(f"Enter the arrival time and burst time for process P{i+1}: ").split())
    processes.append(Process(f"P{i+1}", at, bt))

time_quantum = int(input("Enter the time quantum: "))

round_robin(processes, time_quantum)


Process  Arrival  Burst    Completion   Turnaround   Waiting    Response
P1       0        9        19           19           10         0         
P2       1        4        13           12           8          2         
P3       2        9        22           20           11         4         

Average Turnaround Time: 17.0 ms
Average Waiting Time: 9.666666666666666 ms

Gantt Chart:
0->P1->3->P2->6->P3->9->P1->12->P2->13->P3->16->P1->19->P3->22


In [18]:
#Priority(Pre-emptive)
class Process:
    def __init__(self, name, arrival_time, burst_time, priority):
        self.name = name
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.priority = priority

def calculate_waiting_time(processes, n, waiting_time, turnaround_time, completion_time):
    time = 0
    # list to keep track of remaining burst time of processes
    remaining_time = [process.burst_time for process in processes]
    completed = 0
    sequence = []
    
    while completed != n:
        # finding the process with minimum priority and remaining burst time
        min_priority = 99999
        selected_process = None
        for i in range(n):
            if remaining_time[i] > 0 and processes[i].priority < min_priority and processes[i].arrival_time <= time:
                min_priority = processes[i].priority
                selected_process = i
        
        if selected_process is None:
            time += 1
            continue
        
        # executing the selected process for 1 time unit
        remaining_time[selected_process] -= 1
        
        # updating waiting time of all processes
        for i in range(n):
            if i != selected_process and remaining_time[i] > 0 and processes[i].arrival_time <= time:
                waiting_time[i] += 1
        
        # if the selected process has completed its execution
        if remaining_time[selected_process] == 0:
            completed += 1
            completion_time[selected_process] = time + 1
            turnaround_time[selected_process] = completion_time[selected_process] - processes[selected_process].arrival_time
            if waiting_time[selected_process] < 0:
                waiting_time[selected_process] = 0
        
        sequence.append(processes[selected_process].name)
        time += 1
    
    print("\nGantt Chart")
    print(f"0", end="")
    for i in range(len(sequence)-1):
        if sequence[i] != sequence[i+1]:
            print(f"->{sequence[i]}->{i+1}", end="")
    print(f"->{sequence[-1]}->{len(sequence)}")

# taking input from the user
n = int(input("Enter the number of processes: "))
processes = []
for i in range(n):
    at, bt, priority = map(int, input(f"Enter the arrival time, burst time, and priority for process P{i+1}: ").split())
    processes.append(Process(f"P{i+1}", at, bt, priority))

waiting_time = [0] * n
turnaround_time = [0] * n
completion_time = [0] * n

calculate_waiting_time(processes, n, waiting_time, turnaround_time, completion_time)

# calculating the average waiting time and average turnaround time
avg_waiting_time = sum(waiting_time) / n
avg_turnaround_time = sum(turnaround_time) / n

# printing the table
print("\nProcess\tArrival\tBurst\tPriority\tCompletion\tTurnaround\tWaiting")
for i in range(n):
    print(f"{processes[i].name}\t{processes[i].arrival_time}\t{processes[i].burst_time}\t{processes[i].priority}\t\t{completion_time[i]}\t\t{turnaround_time[i]}\t\t{waiting_time[i]}")

# printing the average waiting time and average turnaround time
print(f"\nAverage waiting time: {avg_waiting_time}")
print(f"Average turnaround time: {avg_turnaround_time}")


Gantt Chart
0->P1->1->P2->5->P1->13->P3->22

Process	Arrival	Burst	Priority	Completion	Turnaround	Waiting
P1	0	9	2		13		13		4
P2	1	4	1		5		4		0
P3	2	9	3		22		20		11

Average waiting time: 5.0
Average turnaround time: 12.333333333333334
